In [8]:
import pandas as pd
from pandas.core.arrays.period import timedelta

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import os

In [9]:
def missing_val_count_by_column (df) :
    nan_count = (df.isnull().sum())
    print(nan_count[nan_count > 0]) 
    
def check_nans (df):
    is_NaN = df. isnull()
    row_has_NaN = is_NaN. any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    return rows_with_NaN

In [10]:
#train = pd.read_excel('./data/train_data/train.xlsx')
#val = pd.read_excel('./data/val_data/validate.xlsx')
train = pd.read_excel('/content/train.xlsx')
val = pd.read_excel('/content/validate.xlsx')

In [11]:
print(train.shape, val.shape)
missing_val_count_by_column(train)
missing_val_count_by_column(val)

(1096, 25) (730, 25)
Series([], dtype: int64)
Series([], dtype: int64)


In [12]:
def create_avg_day(df):
    if 'avg_day' not in df.columns:
        df['avg_day']=df.iloc[:, 1:].mean(axis = 1)
    else:
        print("There's already a column avg_day! No need to create another one.")
        
        
def rename_prices(df):
    if 'PRICES' in df.columns:
        df.rename(columns={"PRICES": "datetime"}, inplace = True)
    else:
        print("There's no column PRICES.")    

In [13]:
create_avg_day(train)
rename_prices(train)

create_avg_day(val)
rename_prices(val)

Date formatter: https://github.com/d3/d3-time-format/tree/v2.2.3#locale_format

# Data Formatting

In [14]:
def dataformatting(df):
    #wide to long
    df = df.melt(id_vars=['datetime'], value_vars=df.columns[1:25]).sort_values(['datetime', 'variable'])
    df.reset_index(inplace=True, drop=True)
    
    #creating master time column, ulgy but works
    time = df['datetime'].copy()
    for d in range(len(df['datetime'])):
        time[d] = df['datetime'][d]+timedelta(hours = d%24) #decided not to go for the +1, so hour 1 is midnight, makes more sense, now it ends in 2009, otherwise the last measurement was 01.01.2010 00:00:00
    df['time'] = time
    
    #hour from string to int
    df['variable'] = df['variable'].map(lambda x:int(x[-2:]))
    
    #renaming, shullfing columns (not important)
    df.rename(columns={"datetime": "date", "variable": "hour", "value":"price"}, inplace = True)

    df['month'] = pd.DatetimeIndex(df['time']).month
    df['day'] = pd.DatetimeIndex(df['time']).day
    df['hour'] = pd.DatetimeIndex(df['time']).hour + 1

    cols = df.columns.tolist()
    cols = [cols[0]] + [cols[3]] + [cols[2]] + [cols[4]] + [cols[5]] + [cols[1]]
    df = df[cols]
    
    return df
    

TO DO:
- make pipeline for big and small dfs
- remove 2 first columns at the end!

# Features

Long Term: 
- (hourly) bollinger bands (5d, 14d, 30d)
- (hourly) EMA (5d, 14d, 30d)
- (daily) ATR (5d, 14d, 30d)
- (hourly) stochastic oscilator (5d + smoothed, 14d + smoothed, 30d + smoothed)

Short Term:
- (hourly) bollinger bands (8h, 24h)
- (hourly) EMA (8h, 24h)
- (hourly) stochastic oscilator (24h + smoothed)

## Bollinger Bands
A Bollinger Band is a technical analysis tool defined by a set of lines plotted two standard deviations (positively and negatively) away from a simple moving average (SMA) of the Stocks’ price Bollinger Bands allow traders to monitor and take advantage of shifts in price volatilities

Main Components of a Bollinger Bands:
 - Upper Band: The upper band is simply two standard deviations above the moving average of a stock’s price.
 - Middle Band: The middle band is simply the moving average of the stock’s price.
 - Lower Band: Two standard deviations below the moving average is the lower band.
 
Bollinger Bands allow traders to monitor and take advantage of shifts in price volatilities.

When calculating the SMA for Bollinger bands, traders typically use a 20 day SMA. Here is how you would calculate the SMA of a stock.

In [15]:
def bollinger_bands(df, hours):
    out = df.copy()
    rolled_mean = out['price'].rolling(hours).mean() 
    rolled_mean.reset_index(inplace=True, drop=True)
    for i in range (1,hours):
        short = out['price'][0:i]
        rolled_mean.loc[i-1] = short.mean()
    
    rolled_std = out['price'].rolling(hours).std() 
    rolled_std.reset_index(inplace=True, drop=True)
    for i in range (1,hours):
        # that's an estimation, first few points are quite bad but evens out quickly
        short = out['price'][0:i+5] #to make it less biased, i+5, so no 0 at the beginning etc
        rolled_std.loc[i-1] = short.std()

    bollinger_up = rolled_mean + rolled_std * 2 # Calculate top band
    bollinger_middle = rolled_mean
    bollinger_down = rolled_mean - rolled_std * 2 # Calculate bottom band

    out[f'{hours}h / {int(hours/24)}day bollinger down'] = bollinger_down
    out[f'{hours}h / {int(hours/24)}day bollinger middle'] = bollinger_middle
    out[f'{hours}h / {int(hours/24)}day bollinger up'] = bollinger_up

    return out

## Exponential moving average (EMA)
An exponential moving average (EMA) is a type of moving average (MA) that places a greater weight and significance on the most recent data points. The exponential moving average is also referred to as the exponentially weighted moving average. An exponentially weighted moving average reacts more significantly to recent price changes than a simple moving average (SMA), which applies an equal weight to all observations in the period.

One of the most popular EMA spans to use are: 8, 12, 20 (for short term trading) and 50, 200 (for long term)

Some traders use Fibonacci numbers (5, 8, 13, 21 ...) to select moving averages.

In general, the 50- and 200-day EMAs are used as indicators for long-term trends. When a stock price crosses its 200-day moving average, it is a technical signal that a reversal has occurred.

In [16]:
def get_ema(df, period_length:int):
    #returns a new df, which contains new column
    x = df.copy()
    x[f'{period_length}h / {int(period_length/24)}day EMA'] = x['price'].ewm(span=period_length).mean()
    return x

## Average true range (ATR)
The average true range (ATR) is a technical analysis indicator that measures market volatility by decomposing the entire range of an asset price for that period. ATR measures market volatility. It is typically derived from the 14-day moving average of a series of true range indicators.

In [17]:
def get_atr(df, period:int):
    x = df.copy()
    maxes = (x.groupby(['date']).max()['price'])
    mins = (x.groupby(['date']).min()['price'])
    closing_prices = df.loc[df['hour'] == 24]['price']

    maxes.reset_index(inplace=True, drop=True)
    mins.reset_index(inplace=True, drop=True)
    closing_prices.reset_index(inplace=True, drop=True)

    yesterday_prices = closing_prices.shift(1)
    yesterday_prices[0] = yesterday_prices[1] #taking care of NA
    
    tr1 = maxes-mins
    tr2 = abs(maxes - yesterday_prices)
    tr3 = abs(mins - yesterday_prices)

    tmp = pd.DataFrame({"A": tr1, "B": tr2, "C":tr3})
    true_range = tmp[["A", "B", "C"]].max(axis=1)
    a_tr = pd.Series([true_range[0]], dtype = np.float64)
    for i in range(1, len(true_range)):
        if i < period:
            a_tr[i] = (a_tr[i-1] * (i-1) + true_range[i]) / i
        else:
            a_tr[i] = (a_tr[i-1] * (period-1) + true_range[i]) / period

    #extending it so it's same for the day
    a_tr = a_tr.loc[a_tr.index.repeat(24)]
    a_tr.reset_index(inplace=True, drop=True)
    x[f'{period*24}h / {int(period)}day ATR'] = a_tr
    return x

# Stochastic Oscillator
ranges 0-100, measure of over/underbuying

In [18]:
def stochastic_osc(df, hours, smoothing):
  out = df.copy()
  maxes = out['price'].rolling(hours).max()
  mins = out['price'].rolling(hours).min()
  sliced = out['price'][0:hours-1]
  mins[0:hours-1] = min(sliced)
  maxes[0:hours-1] = max(sliced)
  oscillator = (out['price'] - mins) / (maxes - mins) * 100
  out[f'{hours}h / {int(hours/24)}stochastic oscillator'] = oscillator
  #usually you use both the oscillator and rolling average of it, this function adds both if bool flag smoothing, hardcoded 8 hours moving avg
  if smoothing:
    osc_smoothed = oscillator.rolling(8).mean()
    #simple NaN handling
    osc_smoothed[0:7] = osc_smoothed[7]
    out[f'{hours}h / {int(hours/24)}smoothed oscillator'] = osc_smoothed
  return out  

In [19]:
def build_big(data_f):
    df = data_f.copy()
    df = dataformatting(df)
    
    df = bollinger_bands(df, 8)
    df = bollinger_bands(df, 24)
    df = bollinger_bands(df, 120)
    df = bollinger_bands(df, 336)
    df = bollinger_bands(df, 720)
    
    df = get_ema(df, period_length=8)
    df = get_ema(df, period_length=24)
    df = get_ema(df, period_length=120)
    df = get_ema(df, period_length=336)
    df = get_ema(df, period_length=720)

    df = get_atr(df,5) #here operating on days!
    df = get_atr(df,14) #here operating on days!
    df = get_atr(df,30) #here operating on days!

    df = stochastic_osc(df, 24, True)
    df = stochastic_osc(df, 120, True)
    df = stochastic_osc(df, 336, True)
    df = stochastic_osc(df, 720, True)

    df.drop(columns = ['date', 'time'], inplace = True)
    
    return df

In [21]:
def build_small(data_f):
    df = data_f.copy()
    df = dataformatting(df)

    df = bollinger_bands(df, 24)

    df = get_ema(df, period_length=24)
    
    df = get_atr(df,5) #here operating on days!
    
    df = stochastic_osc(df, 24, False)
    
    df.drop(columns = ['date', 'time'], inplace = True)
    
    return df

In [23]:
tmp = build_small(train)
tmp.head(10)

,price,month,day,hour,24h / 1day bollinger down,24h / 1day bollinger middle,24h / 1day bollinger up,24h / 1day EMA,120h / 5day ATR,24h / 1stochastic oscillator
0,24.31,1,1,1,0.814399,24.310000,47.805601,24.310000,37.98,63.981043
1,24.31,1,1,2,0.683242,24.310000,47.936758,24.310000,37.98,63.981043
2,21.71,1,1,3,0.168658,23.443333,46.718009,23.370150,37.98,57.135334
3,8.42,1,1,4,-3.050800,19.687500,42.425800,19.153005,37.98,22.143233
4,0.01,1,1,5,-5.741649,15.752000,37.245649,14.660904,37.98,0.000000
5,0.01,1,1,6,-7.266726,13.128333,33.523393,11.683418,37.98,0.000000
6,0.02,1,1,7,-8.196577,11.255714,30.708006,9.573125,37.98,0.026330
7,0.01,1,1,8,-10.157723,9.850000,29.857723,8.001474,37.98,0.000000
8,0.01,1,1,9,-10.614460,8.756667,28.127793,6.790274,37.98,0.000000
9,6.31,1,1,10,-10.342721,8.512000,27.366721,6.722344,37.98,16.587678


Creating DF for tabular learning

In [ ]:
train_discrete_path = os.path.join(os.getcwd(),'data/train_data/train_discrete.npy')
val_discrete_path = os.path.join(os.getcwd(),'data/val_data/val_discrete.npy')

with open(train_discrete_path,'wb') as f: 
    np.save(f,categorical_train.to_numpy())

with open(val_discrete_path,'wb') as f:
    np.save(f,categorical_val.to_numpy())

In [ ]:
#to save to .npy file
np_train = train_preprocessed[train_preprocessed.columns[3:]].to_numpy()
with open('./data/train_data/train_big.npy', 'wb') as f:
    np.save(f, np_train)
    
np_val = val_preprocessed[val_preprocessed.columns[3:]].to_numpy()
with open('./data/val_data/val_big.npy', 'wb') as f:
    np.save(f, np_val)

In [ ]:
#to open .npy file
with open('./data/train_data/train.npy', 'rb') as f:
    np_train = np.load(f)